[Reference](https://towardsdatascience.com/how-to-deploy-machine-learning-models-as-a-microservice-using-fastapi-b3a6002768af)

# Step 1. Make your model for which you want to create the API ready


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import pickle

# Load dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
names = ['sepal-length', 'sepal-width', 'petal-length', 'petal-width', 'class']
dataset = pd.read_csv(filepath_or_buffer=url,header=None,sep=',',names=names)

# Split-out validation dataset
array = dataset.values
X = array[:,0:4]
y = array[:,4]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1, shuffle=True)
classifier = LogisticRegression()
classifier.fit(X_train,y_train)

# save the model to disk
pickle.dump(classifier, open('LRClassifier.pkl', 'wb'))

# load the model from disk
loaded_model = pickle.load(open('LRClassifier.pkl', 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

0.9666666666666667


# Step 2. Create API using FastAPI framework


In [3]:
!pip install fastapi

     |████████████████████████████████| 51 kB 452 kB/s 
     |████████████████████████████████| 61 kB 248 kB/s 
     |████████████████████████████████| 10.1 MB 37.8 MB/s 
     |████████████████████████████████| 78 kB 6.5 MB/s 


In [4]:
from fastapi import FastAPI
from pydantic import BaseModel
import pickle
import numpy as np
import pandas as pd

app = FastAPI()
class IrisSpecies(BaseModel):
    sepal_length: float
    sepal_width: float
    petal_length: float
    petal_width: float

@app.post('/predict')
async def predict_species(iris: IrisSpecies):
    data = iris.dict()
    loaded_model = pickle.load(open('LRClassifier.pkl', 'rb'))
    data_in = [[data['sepal_length'], data['sepal_width'], data['petal_length'], data['petal_width']]]
    prediction = loaded_model.predict(data_in)
    probability = loaded_model.predict_proba(data_in).max()
    return {
    'prediction': prediction[0],
    'probability': probability
    }

In [6]:
import requests
new_measurement = {
"sepal_length": 1.2,
"sepal_width": 2.3,
"petal_length": 1.4,
"petal_width": 2.8
}
response = requests.post('http://127.0.0.1:8000/predict', json=new_measurement)
print(response.content)